<a href="https://colab.research.google.com/github/danielhou13/cogs402longformer/blob/main/cogs402longformer/fakeclinicalnotes/src/cogs402projecttestnotes/AttributionsLongformerNotes2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook adapts the [Captum tutorial for question answering](https://captum.ai/tutorials/Bert_SQUAD_Interpret) and refactors it into the longformer sequence classification task. Specifically, this notebook focuses on using the model's embeddings to get token attributions for the examples of your choice, or the entire dataset if needed. By doing so, we can visualize which tokens have the most influence in the model's prediction, and find out the k tokens with the most influence at helping the model predict correctly as well as incorrectly.

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import dependencies

In [25]:
pip install transformers --quiet

In [26]:
pip install captum --quiet

In [27]:
pip install datasets --quiet

In [28]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [29]:
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

import torch
import pandas as pd

In [30]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Import model

Here we are importing the model and tokenizer and letting the model use our GPU to run. Please change model path, and tokenizer to whichever one you wish to use.

In [31]:
from transformers import LongformerForSequenceClassification, LongformerTokenizer, LongformerConfig
# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model

# load model
test = torch.load("/content/drive/MyDrive/fakeclinicalnotes/models/full_augmented_lr2e-5_dropout3_10_trained_threshold.pt")
model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096', state_dict=test['state_dict'], num_labels = 2)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['longformer_model.encoder.layer.9.attention.self.value_global.weight', 'longformer_model.encoder.layer.11.attention.output.LayerNorm.weight', 'longformer_model.encoder.layer.3.output.LayerNorm.bias', 'longformer_model.encoder.layer.7.attention.output.LayerNorm.weight', 'longformer_model.encoder.layer.1.output.LayerNorm.weight', 'longformer_model.encoder.layer.9.attention.self.key.bias', 'longformer_model.encoder.layer.0.attention.output.dense.weight', 'longformer_model.encoder.layer.4.attention.self.key.bias', 'longformer_model.encoder.layer.5.attention.self.value.weight', 'longformer_model.encoder.layer.2.attention.output.LayerNorm.weight', 'longformer_model.encoder.layer.5.attention.self.query_global.bias', 'longformer_model.encoder.layer.4.attention.output.dense.bias', 'longformer_model.encoder.layer.3.attention.output.dense.bias', 'longformer_mo

Create functions that give us the input ids and the position ids for the text we want to examine along with the baselines for integrated gradients.

In [32]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [33]:
max_length = 2046
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):

    text_ids = tokenizer.encode(text, truncation = True, add_special_tokens=False, max_length = max_length)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)

    #taken from the longformer implementation
    mask = input_ids.ne(ref_token_id).int()
    incremental_indices = torch.cumsum(mask, dim=1).type_as(mask) * mask
    position_ids = incremental_indices.long().squeeze() + ref_token_id

    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    position_ids = position_ids[:, :seq_length]
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

### Import Dataset

Here we import the papers dataset

In [34]:
from datasets import load_dataset
import numpy as np
cogs402_ds = load_dataset("danielhou13/cogs402datafake")["train"]

  0%|          | 0/1 [00:00<?, ?it/s]

Here we import the news dataset

In [35]:
# cogs402_ds = load_dataset("danielhou13/cogs402dataset2")["validation"]

## Getting the Attributions

A custom forward function that returns the softmaxed logits, which are the class probabilities that the model uses for prediction.

In [36]:
def predict(inputs, position_ids=None, attention_mask=None):
    output = model(inputs,
                   position_ids=position_ids,
                   attention_mask=attention_mask)
    return output.logits

In [37]:
#set 1 if we are dealing with a positive class, and 0 if dealing with negative class
def custom_forward(inputs, position_ids=None, attention_mask=None):
    preds = predict(inputs,
                   position_ids=position_ids,
                   attention_mask=attention_mask
                   )
    return torch.softmax(preds, dim = 1)

A helper function to summarize attributions for each word token in the sequence.

In [38]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.linalg.norm(attributions)
    return attributions

Perform Layer Integrated Gradients using the longformer's embeddings.

In [39]:
lig = LayerIntegratedGradients(custom_forward, model.longformer.embeddings)

This function will let us get the example and the baseline inputs in order to perform integrated gradients, and add the attributions to our visualization tool. Additionally, we will add the attributions and tokens for each example into an array so we can use them when we want to further examine the attributions scores for each example. More information about the integrated gradients function can be found [here](https://captum.ai/api/layer.html#layer-integrated-gradients).

In [40]:
vis_data_records = []
all_attributions = {}
all_tokens = {}
all_deltas = {}

In [41]:
# Takes in dataset and example number
def get_token_attributions(dataset, example):
  text = dataset['text'][example]
  label = dataset['labels'][example]
  # get the inputs, position ids, attention mask, and the baselines
  input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
  position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
  attention_mask = construct_attention_mask(input_ids)

  #get the tokens
  indices = input_ids[0].detach().tolist()
  all_tokens_curr = tokenizer.convert_ids_to_tokens(indices)
  all_tokens_curr = [x.replace('Ġ', '') for x in all_tokens_curr]
  all_tokens[str(example)] = all_tokens_curr

  #perform integrated gradients
  attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    return_convergence_delta=True,
                                    additional_forward_args=(position_ids, attention_mask),
                                    target=1,
                                    n_steps=500,
                                    internal_batch_size = 2)

  # We want one value for every token.
  attributions_sum = summarize_attributions(attributions)

  # store the values in our dictionary
  all_attributions[str(example)] = attributions_sum
  all_deltas[str(example)] = attributions_sum

  # get the score for our visualization
  score = predict(input_ids, position_ids, attention_mask)

  # storing couple samples in an array for visualization purposes
  # requires array of attributions, prediction score, predicted class, true class 
  # the label you want your attributions to associate positive with, the attribution score
  # the tokens, and the delta if you have it.
  vis_data_records.append(viz.VisualizationDataRecord(
                        attributions_sum,
                        torch.softmax(score, dim = 1).max(),
                        torch.argmax(torch.softmax(score, dim = 1)),
                        label,
                        str(1),
                        attributions_sum.sum(),       
                        all_tokens_curr,
                        delta)
  )

Here we are taking some examples from the Papers datasets.

In [42]:
get_token_attributions(cogs402_ds, 7)

This function allows us to display our attributions in a manner that is easy to read. We can see the attributions of the word overlayed on top of their respective token. The green colour represents positive attributions (i.e. the model is attributing this token to influential for predicting the positive class) while the red colour represents negative attributions. 

In [43]:
# # storing couple samples in an array for visualization purposes
# score_vis = viz.VisualizationDataRecord(
#                         attributions_sum,
#                         torch.softmax(score, dim = 1).max(),
#                         torch.argmax(torch.softmax(score, dim = 1)),
#                         label,
#                         str(1),
#                         attributions_sum.sum(),       
#                         all_tokens,
#                         delta)

print('\033[1m', 'Visualization For Score', '\033[0m')
_ = viz.visualize_text(vis_data_records)

 Visualization For Score 


## Further Examination of the Attributions

Next we might want to look in-depth about the attribution scores for each token of an example. We saved the attributions for the examples we looked at above, so we can easily retrieve the attributions. We also grab the examples because we want to know what tokens the attributions are associated with.

Both lists are of shape: (seq_len)

In [44]:
example = 7
attributions_sum = all_attributions[f"{example}"]
all_tokens2 = all_tokens[f"{example}"]

These functions return which words had the strongest (most positive and most negative) attributions. Change the number of tokens you wish to visualize for your needs. It takes in the attributions and the tokens we grabbed in the previous cell and returns 3 lists: the topk (or bottomk) attributions, their respective token and their position.

Note: Remember that the attributions are with respect to the positive class, so the most impact tokens that helped the model predict the negative class will be in the botk attributed tokens.

In [45]:
def get_topk_attributed_tokens(attrs, all_tokens, k=20):
    values, indices = torch.topk(attrs, k)
    top_tokens = [all_tokens[idx] for idx in indices]
    return top_tokens, values, indices

In [46]:
def get_botk_attributed_tokens(attrs, all_tokens, k=20):
    values, indices = torch.topk(attrs, k, largest=False)
    top_tokens = [all_tokens[idx] for idx in indices]
    return top_tokens, values, indices

Convert the values, index of the values, and the token into a pandas Dataframe for visualization. It will be sorted by highest value for attributions to lowest. Alternatively, if youre looking for the most negative attributions, it goes from lowest to highest.

In [47]:
top_words_start, top_words_val_start, top_word_ind_start = get_topk_attributed_tokens(attributions_sum.cpu(), all_tokens2)
bot_words_start, bot_words_val_start, bot_word_ind_start = get_botk_attributed_tokens(attributions_sum.cpu(), all_tokens2)

df_high = pd.DataFrame({'Word': top_words_start, 'index':top_word_ind_start, 'attribution': top_words_val_start})

df_low = pd.DataFrame({'Word': bot_words_start, 'index':bot_word_ind_start, 'attribution': bot_words_val_start})
# df_start.style.apply(['cell_ids: False'])

# ['{}({})'.format(token, str(i)) for i, token in enumerate(all_tokens)]

Here we display our top k positively and negatively attributed tokens for our example.

In [48]:
df_high

,Word,index,attribution
0,cul,1112,0.069536
1,outpatient,1924,0.063631
2,cul,1347,0.057573
3,are,290,0.054632
4,are,1734,0.054299
5,l,187,0.053642
6,matter,768,0.053257
7,equipment,481,0.053209
8,DIS,43,0.053008
9,DIS,319,0.052307


In [49]:
df_low

,Word,index,attribution
0,Date,103,-0.072768
1,ax,280,-0.065516
2,Mel,171,-0.064518
3,added,247,-0.059923
4,ax,163,-0.059533
5,Date,88,-0.058685
6,**,1256,-0.057988
7,standard,1727,-0.057642
8,ox,288,-0.056908
9,protective,1789,-0.056630


In [50]:
d = {"tokens":all_tokens2, "attribution":attributions_sum[:len(all_tokens2)].cpu()}

We notice that there are many repeating tokens in each example that have different positions. While we might want to know how the position plays into the attributions, if we want to know strictly based on the tokens itself, we can add all the duplicate tokens together to get the aggregate attribution for each token. Therefore, we aggregate the attributions strictly based on token type.

In [51]:
df_attrib = pd.DataFrame(d)
aggregation_functions = {'attribution': 'sum'}
df_new = df_attrib.groupby(df_attrib['tokens']).aggregate(aggregation_functions)

In [52]:
highest_attrib_tokens = df_new.sort_values(by=['attribution'], ascending=False).reset_index()
highest_attrib_tokens[:10]

,tokens,attribution
0,No,0.751348
1,and,0.583351
2,",",0.450273
3,one,0.367632
4,are,0.324038
5,cul,0.262463
6,respect,0.238792
7,with,0.232330
8,world,0.212465
9,from,0.201487


In [53]:
lowest_attrib_tokens = df_new.sort_values(by=['attribution']).reset_index()
lowest_attrib_tokens[:10]

,tokens,attribution
0,.,-2.737409
1,,-2.184417
2,**,-0.744449
3,was,-0.731146
4,to,-0.728154
5,a,-0.665223
6,ayne,-0.517481
7,per,-0.481157
8,or,-0.399242
9,very,-0.349094


## Masking the stopwords and non-alpha tokens

There may be some stopwords or punctuations in our top attributed tokens, so now that we have the list of the highest and lowest, we can identify interesting keywords.

In [54]:
import nltk
from transformers import AutoTokenizer
nltk.download('stopwords')
tokenizer2 = AutoTokenizer.from_pretrained('allenai/longformer-base-4096', add_prefix_space=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [55]:
from nltk.corpus import stopwords
all_stopwords = stopwords.words('english')
all_stopwords.append(" ")
stopwords = set(tokenizer2.tokenize(all_stopwords, is_split_into_words =True))
stopwords.update(all_stopwords)
print(stopwords)

{"shan't", 'Ġwho', 'between', 'Ġbetween', 'Ġof', 'Ġsuch', 'doesn', 'can', 'Ġbe', 'or', 'you', 'Ġwhen', "you'd", 'so', 'him', 'Ġinto', 'his', "mustn't", 'about', 'with', 'above', 'where', 'Ġbut', 'Ġan', 'Ġabove', 'and', 'too', 'Ġll', 'Ġtheirs', 'the', 'does', 'Ġeach', 'how', 'up', 'in', 'shouldn', 'Ġ', 'Ġunder', 'Ġo', 'all', 'Ġhaving', 'me', 'Ġdidn', "'ve", 'Ġy', 'Ġsh', 'Ġmost', 'Ġwhile', 'itself', 'himself', 'Ġours', 'Ġwith', 'Ġuntil', 'Ġdoesn', 'Ġhers', 'Ġthat', 'Ġnor', 'Ġshouldn', 'Ġmyself', 'while', 'both', ' ', 'Ġfrom', 'Ġneed', 'Ġno', 'Ġdoes', "'s", 'Ġsame', 'herself', 'Ġher', 'most', 'Ġthere', 'Ġboth', 'Ġyourself', 'Ġother', 'nor', 'Ġshould', 'Ġsome', 'yourselves', 'Ġto', 't', 'Ġbeen', 'by', 'Ġweren', "'ll", 'Ġwere', 'y', "doesn't", "weren't", 'Ġthese', 'Ġme', "hasn't", 'ma', 'Ġdon', 'but', 'Ġyourselves', 'Ġtheir', 'll', 'Ġand', 'Ġt', 'i', 'out', "you're", 'that', 'Ġhimself', 'Ġthem', 'Ġmy', 'will', 'any', 'Ġonly', 'am', 'has', "'re", 'few', 've', 'Ġany', 'there', 'Ġthis', 'wasn'

Lets see the per-token attributions again

In [56]:
df_high[(df_high['Word'].str.isalpha()) & ~(df_high['Word'].isin(stopwords))][:10].reset_index(drop=True)

,Word,index,attribution
0,cul,1112,0.069536
1,outpatient,1924,0.063631
2,cul,1347,0.057573
3,l,187,0.053642
4,matter,768,0.053257
5,equipment,481,0.053209
6,DIS,43,0.053008
7,DIS,319,0.052307
8,No,457,0.052196
9,No,1229,0.051275


In [57]:
df_low[(df_low['Word'].str.isalpha()) & ~(df_low['Word'].isin(stopwords))][:10].reset_index(drop=True)

,Word,index,attribution
0,Date,103,-0.072768
1,ax,280,-0.065516
2,Mel,171,-0.064518
3,added,247,-0.059923
4,ax,163,-0.059533
5,Date,88,-0.058685
6,standard,1727,-0.057642
7,ox,288,-0.056908
8,protective,1789,-0.056630
9,children,677,-0.056446


Here we have the aggregate attributions for the example

In [58]:
highest_attrib_tokens[(highest_attrib_tokens['tokens'].str.isalpha()) & ~(highest_attrib_tokens['tokens'].isin(stopwords))][:10].reset_index(drop=True)

,tokens,attribution
0,No,0.751348
1,one,0.367632
2,cul,0.262463
3,respect,0.238792
4,world,0.212465
5,parents,0.176133
6,DIS,0.105315
7,people,0.103351
8,risk,0.102171
9,D,0.097235


In [59]:
lowest_attrib_tokens[(lowest_attrib_tokens['tokens'].str.isalpha()) & ~(lowest_attrib_tokens['tokens'].isin(stopwords))][:10].reset_index(drop=True)

,tokens,attribution
0,ayne,-0.517481
1,per,-0.481157
2,ris,-0.314936
3,I,-0.304227
4,ĉ,-0.230790
5,online,-0.141039
6,ric,-0.133658
7,Date,-0.131453
8,autistic,-0.126390
9,ax,-0.125050


Using this [notebook](https://colab.research.google.com/drive/1lktilbL1IY4nBanlzCdP8TLsBNfUsl_U?usp=sharing), we can get the files to view the aggregated attributions for the entire dataset for both the positive and negative classes. This means we summed up and averaged the attributions for every instance of any given token throughout the entire dataset (whether or not they have positive or negative attributions).

In [60]:
# df_word = pd.read_csv("/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/longformer_emb_papers.csv")

Here we see the highest attributions for the positive class, meaning that these tokens have the most influence when the model tries to predict positive. All of these words do have relevence to A.I. related topics.

In [61]:
# df_word[:15]

NameError: ignored

Here we see the largest attributions for the negative class, meaning that these tokens have the most influence when the model predicts negative.

In [ ]:
# df_word[:-15:-1]